In [1]:
import numpy as np
from pymatgen.core import Structure, Lattice, PeriodicSite, DummySpecie
from pymatgen.analysis.graphs import StructureGraph
from pymatgen.analysis.local_env import MinimumDistanceNN, CrystalNN 

In [70]:
import time

In [66]:
defective_struct = Structure.from_file("high_density_defects/BP_spin_500/BPcifs/P_P126N9_2c43a131-a32a-4372-80ed-cb03ec5f2464.cif")
struct_lattice = defective_struct.lattice

ref_unit_cell = Structure.from_file("unit_cells/P.cif")
reference_struct = ref_unit_cell.make_supercell([6,6,1])


def struct_to_dict(structure):
    list_of_sites = structure.sites
    list_of_frac_coords = np.round(structure.frac_coords,3)
    structure_dict = {i: j for i, j in zip(list_of_sites, list_of_frac_coords)}
    return structure_dict

# Convert structures to dictionaries
defective_dict = struct_to_dict(defective_struct)
reference_dict = struct_to_dict(reference_struct)

/home/adamkim/anaconda3/envs/sample001/lib/python3.13/site-packages/pymatgen/core/structure.py:3087: EncodingWarning: We strongly encourage explicit `encoding`, and we would use UTF-8 by default as per PEP 686
  with zopen(filename, mode="rt", errors="replace") as file:
/home/adamkim/anaconda3/envs/sample001/lib/python3.13/site-packages/pymatgen/io/cif.py:1290: UserWarning: Issues encountered while parsing CIF: 23 fractional coordinates rounded to ideal values to avoid issues with finite precision.
  warnings.warn("Issues encountered while parsing CIF: " + "\n".join(self.warnings))


## `Method 1`

In [71]:
def get_defect_structure(defective_dict, reference_dict, struct_lattice): 
    defect_site = []

    # Handle subtitution
    for ref_site, ref_coords in reference_dict.items():
        for def_site, def_coords in defective_dict.items():
            if np.array_equal(ref_coords, def_coords):
                if ref_site.specie != def_site.specie:  # Substitution case
                    defect_site.append(PeriodicSite(
                        species= def_site.species,
                        coords= def_site.frac_coords,
                        coords_are_cartesian=False,
                        lattice= struct_lattice,
                        properties= {"original_new_am": (ref_site.specie.Z, def_site.specie.Z)}
                    ))

    # Handle vacancy
    for ref_site, ref_coords in reference_dict.items():
        if any(np.array_equal(ref_coords, def_coords) for def_coords in defective_dict.values()):
            pass
        else:
            defect_site.append(PeriodicSite(
                species=DummySpecie(),
                coords = ref_coords,
                coords_are_cartesian=False,
                lattice = struct_lattice,
                properties = {"original_new_am": (ref_site.specie.Z, 0)}
                ))

    # Create a structure with the defect sites
    defect_struct = Structure.from_sites(defect_site)
    return defect_struct

start = time.perf_counter()
get_defect_structure(defective_dict, reference_dict, struct_lattice)
end = time.perf_counter()

print(end-start)


0.35297653099951276


## `Method 2`

In [73]:
def get_defect_structure(defective_dict, reference_dict, struct_lattice): 
    defect_site = []

    # Handle subtitution
    for ref_site, ref_coords in reference_dict.items():
        matching = False
        for def_site, def_coords in defective_dict.items():
            if np.array_equal(ref_coords, def_coords):
                matching = True
                if ref_site.specie != def_site.specie:  # Substitution case
                    defect_site.append(PeriodicSite(
                        species= def_site.species,
                        coords= def_site.frac_coords,
                        coords_are_cartesian=False,
                        lattice= struct_lattice,
                        properties= {"original_new_am": (ref_site.specie.Z, def_site.specie.Z)}
                    ))

        if not matching:
            defect_site.append(PeriodicSite(
                species=DummySpecie(),
                coords = ref_coords,
                coords_are_cartesian=False,
                lattice = struct_lattice,
                properties = {"original_new_am": (ref_site.specie.Z, 0)}
                ))

    # Create a structure with the defect sites
    defect_struct = Structure.from_sites(defect_site)
    return defect_struct

start = time.perf_counter()
get_defect_structure(defective_dict, reference_dict, struct_lattice)
end = time.perf_counter()

print(end-start)


0.28800913100167236
